In [1]:
import myfm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MultiLabelBinarizer
from sklearn import metrics
from myfm.utils.benchmark_data import MovieLens100kDataManager

In [2]:
data_manager=MovieLens100kDataManager()

In [13]:
ratings = data_manager.load_rating_all().drop(columns=['timestamp'])
movies = data_manager.load_movie_info()
users = data_manager.load_user_info()

In [14]:
seen = ratings[ratings['user_id'] == 1][['movie_id']]

In [34]:
unseen_movie = movies[~movies['movie_id'].isin(seen['movie_id'].values)].reset_index().drop(columns=['index'])
unseen_movie

,movie_id,title,release_date,unk,url,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genres
0,273,Heat (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Heat%20(1995),0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,Action|Crime|Thriller
1,274,Sabrina (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Sabrina%20(1995),0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Comedy|Romance
2,275,Sense and Sensibility (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Sense%20and%2...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Drama|Romance
3,276,Leaving Las Vegas (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Leaving%20Las...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Drama|Romance
4,277,Restoration (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Restoration%2...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,1678,Mat' i syn (1997),1998-02-06,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama
1406,1679,B. Monkey (1998),1998-02-06,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,Romance|Thriller
1407,1680,Sliding Doors (1998),1998-01-01,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Drama|Romance
1408,1681,You So Crazy (1994),1994-01-01,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Comedy


In [35]:
user = users[users['user_id'] == 1].reset_index().drop(columns=['index'])
user

,user_id,age,gender,occupation,zipcode
0,1,24,M,technician,85711


In [44]:
for i in range(unseen_movie.values.shape[0]-1):
    user.loc[len(user.index)] = user.loc[0]

In [45]:
user

,user_id,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,1,24,M,technician,85711
2,1,24,M,technician,85711
3,1,24,M,technician,85711
4,1,24,M,technician,85711
...,...,...,...,...,...
1405,1,24,M,technician,85711
1406,1,24,M,technician,85711
1407,1,24,M,technician,85711
1408,1,24,M,technician,85711


In [46]:
pd.concat([user,unseen_movie],axis=1, copy=True,)

,user_id,age,gender,occupation,zipcode,movie_id,title,release_date,unk,url,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genres
0,1,24,M,technician,85711,273,Heat (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Heat%20(1995),...,0,0,0,0,0,0,1,0,0,Action|Crime|Thriller
1,1,24,M,technician,85711,274,Sabrina (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Sabrina%20(1995),...,0,0,0,0,1,0,0,0,0,Comedy|Romance
2,1,24,M,technician,85711,275,Sense and Sensibility (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Sense%20and%2...,...,0,0,0,0,1,0,0,0,0,Drama|Romance
3,1,24,M,technician,85711,276,Leaving Las Vegas (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Leaving%20Las...,...,0,0,0,0,1,0,0,0,0,Drama|Romance
4,1,24,M,technician,85711,277,Restoration (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Restoration%2...,...,0,0,0,0,0,0,0,0,0,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,1,24,M,technician,85711,1678,Mat' i syn (1997),1998-02-06,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,...,0,0,0,0,0,0,0,0,0,Drama
1406,1,24,M,technician,85711,1679,B. Monkey (1998),1998-02-06,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,...,0,0,0,0,1,0,1,0,0,Romance|Thriller
1407,1,24,M,technician,85711,1680,Sliding Doors (1998),1998-01-01,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),...,0,0,0,0,1,0,0,0,0,Drama|Romance
1408,1,24,M,technician,85711,1681,You So Crazy (1994),1994-01-01,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,...,0,0,0,0,0,0,0,0,0,Comedy


In [3]:
ratings = data_manager.load_rating_all().drop(columns=['timestamp'])
users = data_manager.load_user_info()
columns_users = []
for col in users.columns:
    if str(col) != 'user_id':
        columns_users.append('user_'+str(col))
    else:
        columns_users.append(str(col))
users = pd.DataFrame(users.values, columns=columns_users)
# Merge tập ratings và users được gom nhóm bởi user_id
result = pd.merge(ratings, users, on="user_id", how="outer")

movies = data_manager.load_movie_info()
columns_movies = []
for col in movies[['movie_id', 'release_date', 'genres']].columns:
    if str(col) != 'movie_id':
        columns_movies.append('movie_'+str(col))
    else:
        columns_movies.append(str(col))
movies = pd.DataFrame(
    movies[['movie_id', 'release_date', 'genres']].values, columns=columns_movies)
result = pd.merge(result, movies, on="movie_id", how="outer")

In [4]:
result

,user_id,movie_id,rating,user_age,user_gender,user_occupation,user_zipcode,movie_release_date,movie_genres
0,196,242,3,49,M,writer,55105,1997-01-24,Comedy
1,305,242,5,23,M,programmer,94086,1997-01-24,Comedy
2,6,242,4,42,M,executive,98101,1997-01-24,Comedy
3,234,242,4,60,M,retired,94702,1997-01-24,Comedy
4,63,242,3,31,M,marketing,75240,1997-01-24,Comedy
...,...,...,...,...,...,...,...,...,...
99995,863,1679,3,17,M,student,60089,1998-02-06,Romance|Thriller
99996,863,1678,1,17,M,student,60089,1998-02-06,Drama
99997,863,1680,2,17,M,student,60089,1998-01-01,Drama|Romance
99998,896,1681,3,28,M,writer,91505,1994-01-01,Comedy


In [12]:
result['movie_release_date'].isnull().values.any()

True

In [ ]:
df = pd.DataFrame(result)
df.to_csv('newResult2.csv', sep=',', encoding='utf-8',index=False)

In [5]:

# print(dataset.load_rating_predefined_split(3))
ratings = dataset.load_rating_all()
users = dataset.load_user_info()
#Merge tập ratings và users được gom nhóm bởi user_id
result =  pd.merge(ratings, users, on="user_id", how="outer")
#Them giới tính vào tập ratings
newResultGender = result.replace({'M':1,'F':0})
newResultGender


,user_id,movie_id,rating,timestamp,age,gender,occupation,zipcode
0,196,242,3,1997-12-04 15:55:49,49,1,writer,55105
1,196,393,4,1997-12-04 16:11:03,49,1,writer,55105
2,196,381,4,1997-12-04 16:08:48,49,1,writer,55105
3,196,251,3,1997-12-04 16:01:14,49,1,writer,55105
4,196,655,5,1997-12-04 16:09:53,49,1,writer,55105
...,...,...,...,...,...,...,...,...
99995,941,919,5,1997-09-23 21:08:07,20,1,student,97229
99996,941,273,3,1997-09-23 21:10:38,20,1,student,97229
99997,941,1,5,1997-09-23 21:12:24,20,1,student,97229
99998,941,294,4,1997-09-23 21:02:12,20,1,student,97229


In [6]:
#lọc ra các nghê nghiệp của user
occupation = users['occupation'].unique()
occupation

array(['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor'],
      dtype=object)

In [7]:
#Chuyển các nghề nghiệp thành dạng số
newOccupation = {occupation[i]: i for i in range(len(occupation))} 
newOccupation

{'technician': 0,
 'other': 1,
 'writer': 2,
 'executive': 3,
 'administrator': 4,
 'student': 5,
 'lawyer': 6,
 'educator': 7,
 'scientist': 8,
 'entertainment': 9,
 'programmer': 10,
 'librarian': 11,
 'homemaker': 12,
 'artist': 13,
 'engineer': 14,
 'marketing': 15,
 'none': 16,
 'healthcare': 17,
 'retired': 18,
 'salesman': 19,
 'doctor': 20}

In [8]:
#Thêm các nghề nghiệp vừa được số hóa vào tập dữ liệu result
newResultJobs  = result.replace(newOccupation)
# newResultJobs[newResultJobs['occupation']==20]  
newResultJobs

,user_id,movie_id,rating,timestamp,age,gender,occupation,zipcode
0,196,242,3,1997-12-04 15:55:49,49,M,2,55105
1,196,393,4,1997-12-04 16:11:03,49,M,2,55105
2,196,381,4,1997-12-04 16:08:48,49,M,2,55105
3,196,251,3,1997-12-04 16:01:14,49,M,2,55105
4,196,655,5,1997-12-04 16:09:53,49,M,2,55105
...,...,...,...,...,...,...,...,...
99995,941,919,5,1997-09-23 21:08:07,20,M,5,97229
99996,941,273,3,1997-09-23 21:10:38,20,M,5,97229
99997,941,1,5,1997-09-23 21:12:24,20,M,5,97229
99998,941,294,4,1997-09-23 21:02:12,20,M,5,97229


In [9]:
#tập dữ liệu sau khi thêm và mã hóa cả giới tính và nghê nghiệp
newReplace = {'M':1,'F':0} | newOccupation
newResult = result.replace(newReplace)
newResult


,user_id,movie_id,rating,timestamp,age,gender,occupation,zipcode
0,196,242,3,1997-12-04 15:55:49,49,1,2,55105
1,196,393,4,1997-12-04 16:11:03,49,1,2,55105
2,196,381,4,1997-12-04 16:08:48,49,1,2,55105
3,196,251,3,1997-12-04 16:01:14,49,1,2,55105
4,196,655,5,1997-12-04 16:09:53,49,1,2,55105
...,...,...,...,...,...,...,...,...
99995,941,919,5,1997-09-23 21:08:07,20,1,5,97229
99996,941,273,3,1997-09-23 21:10:38,20,1,5,97229
99997,941,1,5,1997-09-23 21:12:24,20,1,5,97229
99998,941,294,4,1997-09-23 21:02:12,20,1,5,97229


In [20]:
# Xuất file csv để test import
df = pd.DataFrame(newResult)
df.to_csv('newResult.csv', sep=',', encoding='utf-8',index=False)

In [21]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from tkinter import filedialog as fd
import pyodbc
from pathlib import Path
import pandas as pd
import threading

DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = 'HOLAKAKA-PC'  # SELECT @@SERVERNAME
DATABASE_NAME = 'datasource'
DATABASE_UID = 'nckh'
DATABASE_PWD = '123456'

# ------------------------------CONNECT DATABASE-------------------------------
connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    UID={DATABASE_UID};
    PWD={DATABASE_PWD};
    Trus_Connection = yes;
"""
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# ------------------------------FUNCTION-------------------------------
def run_report(root, *args):
    import_file(*args)
    root.event_generate("<<PhishDoneEvent>>") # This is using tkinter in a thread, but some tkinter methods are ok to use in threads

def create_table(table_name,data):
    create_value = 'CREATE TABLE ' + table_name + ' ('
    first = True
    for col in data.columns:
        if first:
            first = False
        else:
            create_value += ', '

        type_col = col + ' varchar(100)'
        create_value += type_col
        
    
    create_value += ')' 
    cursor.execute(create_value)
    conn.commit()            

def update_table(table_name,data):
    # Set a counter, since there can't be more than 1000 inserts at a time
    counter = 0

    # Create the header row
    values_index = 'VALUES ('
    header_row = 'INSERT INTO ' + table_name + ' ('
    first = True
    for col in data.columns:
        if first:
            first = False
        else:
            header_row += ', '
            values_index += ','
        header_row += col
        values_index += '?'
    
    header_row += ') ' + values_index + ')'

    # Insert DataFrame to Table
    for row in data.itertuples(index=False, name=None): 
        cursor.execute(header_row,tuple(row))
        if counter % 1000 == 0:
            conn.commit()
        # Increase counter
        counter += 1     
    conn.commit() 

def import_file(*args):
    # file type
    filetypes = (
        ('text files', '*.csv'),
        ('All files', '*.*')
    )

    filenames = fd.askopenfilenames(
        title='Open files',
        initialdir='/',
        filetypes=filetypes)    
    path=''.join(filenames) #Convert Filenames to Str
    
    # Import CSV
    data = pd.read_csv (path)   
    df = pd.DataFrame(data)
    table_name = Path(filenames[0]).stem

    # Create table befor add new data
    create_table(table_name,df)
    update_table(table_name,df)


class main:
    def __init__(self, root):
        root.title('Connect Database')
        root.resizable(False, False)
        self.window_height = 530
        self.window_width = 800
        """ gets the coordinates of the center of the screen """

        self.screen_width = root.winfo_screenwidth()
        self.screen_height = root.winfo_screenheight()
        # Coordinates of the upper left corner of the window to make the window appear in the center
        x_cordinate = int((self.screen_width/2) - (self.window_width/2))
        y_cordinate = int((self.screen_height/2) - (self.window_height/2))
        # Open root windown center
        root.geometry("{}x{}+{}+{}".format(self.window_width,
                    self.window_height, x_cordinate, y_cordinate))

        self.single_addnew = None
        # Create button Add new
        btt_addnew = Button(root, text='Add new data',
                            command=self.open_addnew, height=4, width=15)
        btt_addnew.grid(row=1, column=0, sticky=W, padx=10, pady=10)
    
    #   Showlist is tesings, don't use
    def showlist(self):
        list_table=[]
        cursor.execute("Show tables;")
 
        myresult = cursor.fetchall()
        
        for x in myresult:
            print(x)
        
        variable = StringVar(root)
        variable.set(list_table[0]) # default value

        w = OptionMenu(root, variable, *list_table)
        w.grid()
        w = OptionMenu(root, variable, "one", "two", "three")
        w.grid()

    def close_addnew(self):
        self.single_addnew
        self.single_addnew.destroy()
        self.single_addnew = None

     
    def open_addnew(self):
        self.single_addnew
        if self.single_addnew is None:
            self.single_addnew = Toplevel(root)
            self.single_addnew.geometry("250x250")
            self.single_addnew.title("Add new data")
            # open file button
            open_button = ttk.Button(
                self.single_addnew,
                text='Open a File',
                command=self.do_reports
            )

            open_button.grid(column=0, row=1, sticky='w', padx=10, pady=10)    
            root.bind("<<PhishDoneEvent>>", self.report_done)
            # assign to closing button [X]
            self.single_addnew.protocol("WM_DELETE_WINDOW", self.close_addnew)
        else:
            print("Add new already exists")
    
    # Show process bar when import bigdata in SQL
    def do_reports(self, *args):
        self.pbar = ttk.Progressbar(self.single_addnew, orient = HORIZONTAL, mode = 'indeterminate')
        self.pbar.grid(row = 4, column = 3, sticky = (W, E))
        start, end = 4,5
        t1 = threading.Thread(target = run_report, args = [root, start, end])
        t1.start()
        self.pbar.start()

    # Show alert when Import is done
    def report_done(self, event=None):
        self.pbar.stop()
        messagebox.showinfo("Alert", "Your file has been imported successfully!!!")
        self.close_addnew()
    


# ------------------------------MAIN-------------------------------

root = Tk()
main(root)
root.mainloop()
